In [1]:
%reset -f
%clear

import torch
import torchvision
import numpy
import copy
import tqdm
import os

from utils.model_util import LeNet5
from utils.data_util import *
from utils.train_util import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cuda


In [2]:
save_path = './data/dealed-data/'

dataset = 'mnist'
file_path = save_path + 'train_dataset_splited_' + dataset + '.pt'
if os.path.exists(file_path):
    train_dataset_splited = torch.load(file_path)
    print('file existed.')
else:
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    train_dataset, test_dataset = get_dataset(dataset)
    train_dataset_splited = split_data(train_dataset)
    torch.save(train_dataset_splited, file_path)

file existed.


In [3]:
alpha = 0.8
beta = 0.2

num_all_client = 6
num_all_server = 2
all_client = [i for i in range(num_all_client)]
num_server_commu = 3
num_client_commu = 3
num_client_train = 3

server_1_client = [0, 1, 2]
server_2_client = [3, 4, 5]

idx_client_target = train_data_split(train_dataset_splited, all_client)
all_target = train_dataset_splited.keys()

In [4]:
train_dataset_client = []
for client in all_client:
    train_dataset_client.append([])
for client in all_client:
    for target in all_target:
        train_dataset_client[client].extend(idx_to_dataset(train_dataset_splited[target], idx_client_target[client][target]))

In [ ]:
initial_model = LeNet5(28, 28, 1, 10)
# for client in all_client:
#     trained_model_client = train_model(initial_model, train_dataset_client[client], device)
client_model = []
for client in all_client:
    client_model.append(initial_model)

In [ ]:
for epoch_server_commu in range(num_server_commu):
    if epoch_server_commu == 0:
        loss_function = loss_without_distillation()
    else:
        loss_function = loss_with_distillation(alpha, beta)
    for epoch_client_commu in range(num_client_commu):
        for client in all_client:
            trained_model_client = train_model(
                model=client_model[client],
                dataset=train_dataset_client[client],
                criterion=loss_function,
                device=device,
                epochs=num_client_train)
            client_model[client].append(trained_model_client)
        edge_server_aggeragation()
    edge_server_communicate()

In [5]:
a = train_dataset_client[0]
print(type(a), len(a))

<class 'list'> 9996
